In [2]:
from lark import Lark, Transformer

grammar = r"""
  string: ESCAPED_STRING
  %import common.ESCAPED_STRING
"""
parser = Lark(grammar, start='string')

parser.parse('"Hello world"')

def parse(text):
    print(parser.parse(text).pretty())

parse('"Hello world"')

string	"Hello world"



In [3]:
grammar = r"""
   num: SIGNED_NUM
   SIGNED_NUM: /
    [+-]?(0|[1-9][0-9]*[.]?[0-9]*)  # the digits
   /x  
   %ignore " "
"""

parser = Lark(grammar, start='num')
parse('42')

num	42



In [4]:
grammar = r"""
   ?num: start
   ?start: expr|signed_num
   ?expr: add|sub|mul|signed_num
   ?add: br|add "+" sub| sub | br
   ?sub: br|sub "-" mul|mul | br
   ?mul: br|mul "*" div | div | br
   ?div: br|div "/" div | signed_num | br
   ?br: "("expr")"
   signed_num: /[-]?(0|[1-9][0-9]*[.]?[0-9]*)/
   %ignore " "
"""

parser = Lark(grammar, start='num')
parse('(1+2)*(1+3)/2')

mul
  add
    signed_num	1
    signed_num	2
  div
    add
      signed_num	1
      signed_num	3
    signed_num	2



In [5]:
from lark import Transformer

class MyTransformer(Transformer):
    
    def signed_num(self, value):
        (value, ) = value
        return int(value)

    def add(self, items):
        if len(items) == 1: 
            return items[0]
        return items[0] + items[1]
    
    def sub(self, items):
        if len(items) == 1: 
            return items[0]
        return items[0] - items[1]
        
    def mul(self, items):
        if len(items) == 1: 
            return items[0]
        return items[0] * items[1]
 
    def div(self, items):
        if len(items) == 1: 
            return items[0]
        
        return items[0] / items[1]
 
    

tree = parser.parse("(1+4)/(4-1)*(5+2-4*4)/(8/5-(5-3)-1*9)")
MyTransformer().transform(tree)

1.5957446808510638

In [6]:
# TEST CODE GENERATED BY CHATGPT (creates randomly nested expression trees with optional brackets)

import random

functions = {
    "+": lambda x, y: x + y,
    "-": lambda x, y: x - y,
    "*": lambda x, y: x * y,
    "/": lambda x, y: x / y,
}

precedence = {
    "+": 1,
    "-": 1,
    "*": 2,
    "/": 2,
}

class Node:
    def __init__(self, value, left=None, right=None):
        self.value = value      # operator symbol or number
        self.left = left
        self.right = right

    def is_leaf(self):
        return self.left is None and self.right is None


def random_tree(max_depth=3, p_leaf=0.3):
    """
    Generate a random expression tree.
    """
    if max_depth == 0 or random.random() < p_leaf:
        return Node(random.randint(1, 9))

    op = random.choice(list(functions.keys()))
    left = random_tree(max_depth - 1, p_leaf)
    right = random_tree(max_depth - 1, p_leaf)

    return Node(op, left, right)


def to_string(node, parent_op=None, is_right_child=False):
    if node.is_leaf():
        return str(node.value)

    left_str = to_string(node.left, node.value, False)
    right_str = to_string(node.right, node.value, True)

    expr = f"{left_str}{node.value}{right_str}"

    if parent_op is None:
        return expr

    # Parentheses rules
    if precedence[node.value] < precedence[parent_op]:
        return f"({expr})"

    # Needed for cases like a-(b-c) or a/(b*c)
    if (precedence[node.value] == precedence[parent_op]
        and is_right_child
        and node.value in ("-", "/")):
        return f"({expr})"

    return expr

def evaluate(node):
    if node.is_leaf():
        return node.value

    f = functions[node.value]
    return f(evaluate(node.left), evaluate(node.right))




In [7]:
tree = random_tree(max_depth=4)
expr = to_string(tree)
value = evaluate(tree)

print(expr)
print(value)

5*(2-9)-7-(9/1-8-2*4*(3/2))
-31.0


In [ ]:
epsilon = 0.000000001

all_pass = True
for i in range(100):
    r_tree = random_tree(max_depth=4)
    expr = to_string(r_tree)
    try:
        value = evaluate(r_tree)
    except ZeroDivisionError as err:
        print("Generated function divides by zero, skipping")
        continue
    tree = parser.parse(expr)
    transformer_value = MyTransformer().transform(tree)
    PASS = (transformer_value - value) < epsilon
    if (not PASS):
        all_pass = False
        print(f"FAIL: Expected {value}, got {transformer_value} for {expr}")
if all_pass:
    print("All tests pass!")
else:
    print("FAIL")

FAIL: Expected -38.0, got -32.0 for 9-6+3-(2+4)*7-(2-6/1)
FAIL: Expected -21.155555555555555, got -14.933333333333334 for 5/1-5+4-8*6/5+(9*9-5+8)/(7-4*4)
FAIL: Expected 8.0, got 26.0 for 9+(6*2-4+9-(8-8)/(2/8))
FAIL: Expected -5.384615384615384, got -0.5599999999999999 for 5/((7+8)/(9+5)-6-2+(7-9))
FAIL: Expected 20.5, got 23.5 for (7-5-2+9)/6-(4-8-1*9/(4/8))
FAIL: Expected 74.5, got 88.5 for 2+(6+7)*(2+5)+3*3*(1-2)+(5/2-5+7)
FAIL: Expected 6.083333333333333, got 11.333333333333332 for 7+5+6-(5+9)/2+3/9-7+7/2*2
FAIL: Expected -5.1875, got 22.1875 for (7*3+2)/(6-3-7/7)-3+5+4+3+9/8/(2/3)
FAIL: Expected -0.996875, got 4.0 for 5/5/8*5/(9-8)*(7+1)-1
FAIL: Expected 0.1450777202072539, got 7.0 for (2-1)/(1/7+9-(2+7)/2*2)
FAIL: Expected -0.5, got 0.625 for 1-6/8*5/5*3*(1/6)
FAIL: Expected 12.88888888888889, got 21.46666666666667 for 8/6*(3/5)+1+9+8/3*4
All tests pass!
